In [1]:
library(optparse)
library(gridExtra)
library(tidyverse)
library(GenomicRanges)
library(GenomicFeatures)
library(TxDb.Hsapiens.UCSC.hg19.knownGene)
library(org.Hs.eg.db)
library(rtracklayer)

-- Attaching core tidyverse packages --------------------------------------------------------------------------------------------------------------------------------------------------- tidyverse 2.0.0 --
v dplyr     1.1.3     v readr     2.1.4
v forcats   1.0.0     v stringr   1.5.0
v ggplot2   3.4.4     v tibble    3.2.1
v lubridate 1.9.2     v tidyr     1.3.0
v purrr     1.0.2     
-- Conflicts --------------------------------------------------------------------------------------------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::combine() masks gridExtra::combine()
x dplyr::filter()  masks stats::filter()
x dplyr::lag()     masks stats::lag()
i Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: 'BiocGenerics'


The following objects are masked from 'package:lubridate':

    interse

In [2]:
source("ML_utils.R")

parser <- OptionParser()
parser <- add_option(parser, c("--datasets"), type="character")
parser <- add_option(parser, c("--cancer_types"), type="character")
parser <- add_option(parser, c("--cell_number_filter"), type="integer")
parser <- add_option(parser, c("--tss_fragment_filter"),
                     type="character", default="-1")
parser <- add_option(parser, c("--tissues_to_consider"), 
                     type="character", default="all")
parser <- add_option(parser, c("--ML_model"), type="character")
parser <- add_option(parser, c("--annotation"), 
                     type="character", default="default_annotation")
parser <- add_option(parser, c("--feature_importance_method"), type="character")
parser <- add_option(parser, c("--folds_for_test_set"), type="character")
parser <- add_option(parser, c("--plot_bin_counts"), action="store_true", 
                     default=F)
parser <- add_option(parser, c("--plot_bins_volcano"), action="store_true", 
                     default=F)
parser <- add_option(parser, c("--hundred_kb"), action="store_true", 
                     default=F)

In [3]:
args = parse_args(parser, args =
                    c("--datasets=Tsankov",
                      "--cancer_types=Lung-SCC",
                      "--cell_number_filter=100",
                      "--ML_model=XGB",
                      "--annotation=test_annotation",
                      "--feature_importance_method=permutation_importance",
                      "--folds_for_test_set=1-10"))


In [84]:
cancer_types = args$cancer_types
cancer_types = unlist(strsplit(cancer_types, split = ","))
datasets = unlist(strsplit(args$datasets, split = ","))
datasets = sort(datasets)
cell_number_filter = args$cell_number_filter
tss_fragment_filter = unlist(strsplit(args$tss_fragment_filter, split = ","))
annotation = args$annotation
tissues_to_consider = strsplit(args$tissues_to_consider,  split=",")
ML_model = args$ML_model
feature_importance_method = args$feature_importance_method

folds_for_test_set = args$folds_for_test_set
folds_for_test_set = unlist(strsplit(args$folds_for_test_set, split = "-"))
folds_for_test_set = seq(folds_for_test_set[1], folds_for_test_set[2])
plot_bin_counts = args$plot_bin_counts
hundred_kb = args$hundred_kb
plot_bins_volcano = args$plot_bins_volcano


In [85]:
plot_dist <- function(X, bin_names) {
  df <- data.frame(index = 1:nrow(X), 
                          value = X)
  use_names = seq(1, length(df$index), by=10)
  use_names = append(use_names, nrow(X))
  p = ggplot(df) +
    geom_bar(aes(x=index, y=value),
             stat="identity") +
    xlab("Bin") +
    ylab("Counts") +
    geom_text(x=nrow(df) / 2, y=max(df[["value"]]), 
              aes(label=paste0("n=", sum(df[["value"]]))),
              size=10) +
    scale_x_continuous(name="Row Names", breaks=df$index[use_names], 
                       labels=bin_names[use_names]) +
    theme(axis.text.x = element_text(angle = 45, hjust = 1))
  return(p)
}


In [86]:
cancer_type=cancer_types[1]

In [87]:
scATAC_sources = construct_sources_string(datasets)

In [88]:
df_tsankov = readRDS("../../data/processed_data/count_overlap_data/combined_count_overlaps/finalized_annotation/Tsankov_combined_count_overlaps.rds")

In [89]:
df_rawlins = readRDS("../../data/processed_data/count_overlap_data/combined_count_overlaps/finalized_annotation/Rawlins_fetal_lung_combined_count_overlaps.rds")

In [90]:
df_rawlins = df_rawlins[grep("NE", rownames(df_rawlins)),]

In [98]:
df = rbind(df_tsankov, df_rawlins)

In [99]:
keep = read.csv("../../data/processed_data/chr_keep.csv")[["chr"]]

In [100]:
df = df[, colnames(df) %in% keep]

In [101]:
library(stringr)

In [105]:
df = df[, str_sort(colnames(df), numeric = TRUE)]

In [179]:
plot_list <- list()

In [181]:
for (i in 1:nrow(df)) {
  row_data <- as.numeric(df[i, ])
  row_name <- rownames(df)[i]
  plot_list[[i]] <- func(row_data, row_name)
}

In [180]:
func <- function(row, rowname) {
  df <- tibble(x = seq(1, 2128), y = row)
  p <- ggplot(df, aes(x = x, y = y)) + 
    geom_bar(stat = "identity") + 
    xlab("Bin") + 
    ylab("Counts") + 
    ggtitle(rowname)
  return(p)
}


In [184]:
pdf("my_plots_single_page.pdf", width = 11, height = 8.5) # Adjust size as needed

# Arrange all plots on a single page (adjusting nrow and ncol as needed)
do.call(grid.arrange, c(plot_list, ncol = 5, nrow = 5)) # Adjust ncol and nrow to fit your number of plots

# Close the PDF device
dev.off()

png 
  2